In [1]:
import os
import sys
from django.conf import settings
import numpy as np
import tflearn
import random
import pickle
import json
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from flask import Flask, render_template, request
app = Flask(__name__, template_folder='template')

c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
training_file = ('C:/xampp/htdocs/chatbot/training_data')
data = pickle.load(open(training_file, "rb"))

In [3]:
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [4]:
pengetahuan = ('C:/xampp/htdocs/chatbot/pengetahuan.json')

with open(pengetahuan) as json_data:
    pengetahuan = json.load(json_data)

In [5]:
# load deepneuralnet
input_h = tflearn.input_data(shape=(None, len(train_x[0])))
h2 = tflearn.fully_connected(input_h, 9)
h3 = tflearn.fully_connected(h2, 18)
h4 = tflearn.fully_connected(h3, 18)
h5 = tflearn.fully_connected(h4, 9)

In [6]:
output_h = tflearn.fully_connected(h5, len(train_y[0]), activation='softmax')
output_h_reg = tflearn.regression(output_h)
# Define model dan setup tensorboard
model = tflearn.DNN(output_h_reg, tensorboard_dir='tflearn_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("test : %s" % w)
    return np.array(bag)

In [8]:
# load model yang disimpan
load_model = ('C:/xampp/htdocs/chatbot/./model.tflearn')
model.load(load_model)

context = {}

ERROR_THRESHOLD = 0.25

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:/xampp/htdocs/chatbot/./model.tflearn


In [9]:
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))

    return return_list
def response(sentence, userid='hasnafl', show_details=False):
    results = classify(sentence)

    if results:
        while results:
            for i in pengetahuan['pengetahuan']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details:
                            print('context:', i['context_set'])
                        context[userid] = i['context_set']
                        output_var = str(random.choice(i['responses']))
                        return output_var

                    if not 'context_filter' in i or \
                        (userid in context and 'context_filter' in i and i['context_filter'] == context[userid]):
                        if show_details:
                            print('tag:', i['tag'])
                        context_filter = i.get('context_filter', '')
                        output_var = str(random.choice(i['responses']))
                        return output_var

            results.pop(0)

In [10]:
def dir_last_updated(folder):
    return str(max(os.path.getmtime(os.path.join(root_path, f))
                   for root_path, dirs, files in os.walk(folder)
                   for f in files))
#define app routes
@app.route("/")
def index():
    return render_template("index4.html", last_updated=dir_last_updated('C:/xampp/htdocs/chatbot/static'))

In [11]:
@app.route("/get")
#function for the bot response
def get_bot_response():
    userText = request.args.get('msg')
#     print(userText)
    import mysql.connector
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="",
      database="chatbot"
    )
    mycursor = mydb.cursor()
    a = "A111"
    b = "Hi"
    c = response(userText)
    d = 2
    sql = "INSERT INTO chatbot_response (id_user, user_message, bot_response, id_chat) VALUES (%s, %s, %s, %s)"
    val = (a,userText,c,d)

    mycursor.execute(sql, val)

    mydb.commit()

#     print(mycursor.rowcount, "record inserted.")
    return str(response(userText))

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2020 17:48:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:48:52] "GET /static/stylecss.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:48:52] "GET /img/profile.png HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:48:52] "GET /static/jsscript2.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:48:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:49:35] "GET /img/profile.png HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:49:36] "GET /get?msg=bm+itu+apa%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:49:38] "GET /img/profile.png HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:52:24] "GET /img/profile.png HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:52:24] "GET /get?msg=freight+itu+apa%3F HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:52:26] "GET /img/profile.png HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2020 17:57:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:57:10] "GET /img/pr

In [ ]:
# print(classify('?'))
x = response('hahaha')
print(x)